<a href="https://colab.research.google.com/github/ed-ntim/Dask-tutorial-SA/blob/main/examples/openACC_CUDA_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# OpenACC-CUDA workshop

In [1]:
!pwd

/content


## Activate GPU

- To get access to a GPU, click on the *Runtime* menu and select *Change runtime type*. Choose GPU as a Hardware accelerator. It might take a minute for your notebook to connect to a GPU.
- To check whether a GPU has been connected to your session, run the code cell below with the ``!nvidia-smi`` command by hitting ``SHIFT-ENTER`` on it.

In [2]:
!nvidia-smi

Sun Apr 28 07:29:05 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   60C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## Clone the material

In [3]:
!git clone https://github.com/ENCCS/OpenACC-CUDA-beginners.git

Cloning into 'OpenACC-CUDA-beginners'...
remote: Enumerating objects: 1380, done.
remote: Counting objects: 100% (50/50), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 1380 (delta 30), reused 27 (delta 27), pack-reused 1330
Receiving objects: 100% (1380/1380), 14.09 MiB | 33.48 MiB/s, done.
Resolving deltas: 100% (677/677), done.


In [4]:
!ls

OpenACC-CUDA-beginners	sample_data


## Checkout out **colab_gcc** branch

In [5]:
%cd OpenACC-CUDA-beginners
!git checkout colab_gcc

/content/OpenACC-CUDA-beginners
Branch 'colab_gcc' set up to track remote branch 'colab_gcc' from 'origin'.
Switched to a new branch 'colab_gcc'


In [6]:
!git branch

* colab_gcc
  main


## OpenACC

### Hello World

In [7]:
%cd /content/OpenACC-CUDA-beginners/examples/OpenACC/hello-world/

/content/OpenACC-CUDA-beginners/examples/OpenACC/hello-world


In [8]:
!ls

compile.sh  hello.c  hello.F90	README.md


Compile and run C version

In [9]:
!gcc -fopenacc -o hello_c hello.c

In [11]:
! cat hello.c

#include <stdio.h>
#ifdef _OPENACC
#include <openacc.h>
#endif

int main(void) {
#ifdef _OPENACC
  acc_device_t devtype;
#endif

  printf("Hello world from OpenACC\n");
#ifdef _OPENACC
  devtype = acc_get_device_type();
  printf("Number of available OpenACC devices: %d\n", acc_get_num_devices(devtype));
  printf("Type of available OpenACC devices: %d\n", devtype);
#else
  printf("Code compiled without OpenACC\n");
#endif

  return 0;
}


In [10]:
!./hello_c

Hello world from OpenACC
Number of available OpenACC devices: 1
Type of available OpenACC devices: 2


Compile and run Fortran version

In [12]:
!gfortran -fopenacc -o hello_f hello.F90

In [13]:
!./hello_f

 Hello world from OpenACC
Number of available OpenACC devices: 1
Type of available OpenACC devices: 2


### Vector sum

In [14]:
%cd /content/OpenACC-CUDA-beginners/examples/OpenACC/vector-sum/

/content/OpenACC-CUDA-beginners/examples/OpenACC/vector-sum


In [15]:
!ls

c  F90	README.md  solution


In [16]:
# Pick either C or Fortran:
%cd c
#%cd F90
# Have a look at the solutions:
#%cd solution

/content/OpenACC-CUDA-beginners/examples/OpenACC/vector-sum/c


In [26]:
!make

gcc -g -O3 -fopenacc  sum.c -o sum 


In [23]:
! ls
! cat sum.c

Makefile  sum  sum.c
#include <stdio.h>
#ifdef _OPENACC
#include <openacc.h>
#endif

#define NX 102400

int main(void)
{
    double vecA[NX], vecB[NX], vecC[NX];
    double sum;
    int i;

    /* Initialization of the vectors */
    for (i = 0; i < NX; i++) {
        vecA[i] = 1.0 / ((double) (NX - i));
        vecB[i] = vecA[i] * vecA[i];
    }

    /* TODO:
     * Implement vector addition on device with OpenACC
     * vecC = vecA + vecB
     */ 
#pragma acc data copy (vecA, vecB, vecC)
{
#pragma acc parallel
    {
#pragma acc loop
        for (i = 0; i < NX; i++){
          vecC[i] += vecA[i] * vecB[i];
        }
    }
}     

    sum = 0.0;
    /* Compute the check value */
    for (i = 0; i < NX; i++) {
        sum += vecC[i];
    }
    printf("Reduction sum: %18.16f\n", sum);

    return 0;
}


In [28]:
! ./sum -OPENACC



Reduction sum: 1.2020569031119110


### Double loop

In [ ]:
%cd /content/OpenACC-CUDA-beginners/examples/OpenACC/doubleloop/

In [ ]:
!ls

In [ ]:
%cd c
# Have a look at the solutions:
#%cd solution

### Jacobi

In [ ]:
%cd /content/OpenACC-CUDA-beginners/examples/OpenACC/jacobi/

In [ ]:
!ls

In [ ]:
%cd c
# Have a look at the solutions:
#%cd solution

### $\pi$ Monte Carlo

In [ ]:
%cd /content/OpenACC-CUDA-beginners/examples/OpenACC/pi_monte_carlo/

In [ ]:
!ls

In [ ]:
%cd c
# Have a look at the solutions:
#%cd solution

### Heat equation

In [ ]:
%cd /content/OpenACC-CUDA-beginners/examples/OpenACC/HeatEquation/

In [ ]:
!ls

In [ ]:
%cd c
# Have a look at the solutions:
#%cd solution

### Jacobi

In [ ]:
%cd /content/OpenACC-CUDA-beginners/examples/OpenACC/jacobi/c

In [ ]:
# !gcc -g -O3 -fopenacc  jacobi.c -o jacobi  -lm
!make

In [ ]:
!./jacobi

go to solution

In [ ]:
%cd ../solution/c

In [ ]:
# !gcc -g -O3 -fopenacc  jacobi.c -o jacobi  -lm
!make

### Heat equation

In [ ]:
%cd /content/OpenACC-CUDA-beginners/examples/OpenACC/HeatEquation/c

In [ ]:
!make

In [ ]:
!./heat

go to solution

In [ ]:
%cd ../solution/c

In [ ]:
!gcc -fopenacc -o heat_equation_openacc_1 /content/OpenACC-CUDA-beginners/examples/OpenACC/HeatEquation/solution/c/heat_equation_openacc_1.c

In [ ]:
!./heat_equation_openacc_1

## CUDA

### DeviceList

In [ ]:
%cd /content/OpenACC-CUDA-beginners/examples/CUDA/DeviceList

In [ ]:
!ls

### VectorAdd

In [ ]:
%cd /content/OpenACC-CUDA-beginners/examples/CUDA/VectorAdd/

In [ ]:
!ls

### VectorDotProduct

In [ ]:
%cd /content/OpenACC-CUDA-beginners/examples/CUDA/VectorDotProduct/

In [ ]:
!ls

### MatrixAdd

In [ ]:
%cd /content/OpenACC-CUDA-beginners/examples/CUDA/MatrixAdd/

In [ ]:
!ls

### Heat Equation

In [ ]:
%cd /content/OpenACC-CUDA-beginners/examples/CUDA/HeatEquation/

In [ ]:
!ls